In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
import re
import time
import pymongo
from pymongo import MongoClient


In [2]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]


In [3]:
start_urls_list = {
    'Journals' : 'https://openknowledge.worldbank.org/handle/10986/2210/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Serial Publications': 'https://openknowledge.worldbank.org/handle/10986/11863/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Technical Papers':'https://openknowledge.worldbank.org/handle/10986/18022/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Economic and Sector Work':'https://openknowledge.worldbank.org/handle/10986/6/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en',
    'Working papers':'https://openknowledge.worldbank.org/handle/10986/8/discover?rpp=10&etal=0&group_by=none&page=1&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en',
    'Knowledge Notes':'https://openknowledge.worldbank.org/handle/10986/9387/discover?filtertype=supportedlanguage&filter_relational_operator=equals&filter=en'
}

In [4]:
def add_to_dict(arg):
    try:
        arg
    except NoSuchElementException:
        pass


def get_all_elements_on_page(driver):
    try:
        title = driver.find_element_by_xpath("//*[@class='ds-div-head']").text
    except NoSuchElementException:
        title = ''
        pass
    try:
        abstract = driver.find_element_by_xpath("//*[@class='okr-item-page-field-wrapper abstract']").text
    except NoSuchElementException:
        abstract = ''
        pass 
    try:   
        abstract_views = driver.find_element_by_xpath("//*[@class='item-stat abstract-views']").text.split("\n")[1]
    except NoSuchElementException:
        abstract_views = ''
        pass
    try:
        file_downloads = driver.find_element_by_xpath("//*[@class='item-stat file-downloads']").text.split("\n")[1]
    except NoSuchElementException:
        file_downloads = ''
        pass
    try:
        published_date = driver.find_element_by_xpath("//*[@class='simple-item-view-other word-break']").text
    except NoSuchElementException:
        published_date = ''
        pass
    try:
        pdf_link = driver.find_element_by_xpath("//*[@class='bitstream-link']").get_attribute('href')
    except NoSuchElementException:
        pdf_link = ''
        pass
    
    dict_document = {'title':title,'abstract':abstract,'abstract_views':abstract_views,
                     'file_downloads':file_downloads, 'published_date' : published_date, 'pdf_link':pdf_link,
                     'collection':''}
    return dict_document
    


In [7]:
options = Options()
options.headless = True
driver = webdriver.Firefox()

collection = 'Knowledge Notes'
temp_url = 'https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=159&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en'
driver.get(temp_url)
#driver.get(start_urls_list[collection])
time.sleep(3)

## Find number pages in collection
last_page_link = driver.find_element_by_xpath("//*[@class='last-page-link']")
child_element_split_url = last_page_link.find_elements_by_xpath(".//*")[0].get_attribute('href').split('&')
for element in child_element_split_url:
    if 'page' in element:
        total_pages_numb = int(element.split("=")[1])


print(total_pages_numb)
n=0
for results_page in range(total_pages_numb-1):
    n+=1
    print(n)
    button_next_page = driver.find_element_by_xpath("//*[@class='next-page-link']").get_attribute('href')
    print(button_next_page)
    # get list links on page
    items_on_page = driver.find_elements_by_xpath("//*[@class='item-wrapper']")
    root_url = 'https://openknowledge.worldbank.org/'
    lst_links_documents = [root_url + element.get_attribute('data-href') for element in items_on_page]

    for document_link in lst_links_documents:
        driver.get(document_link)
        time.sleep(3)
        dict_infos = get_all_elements_on_page(driver)
        dict_infos['collection'] = collection
        reports_coll.insert_one(dict_infos)
    # move to next page
    driver.get(button_next_page)
    time.sleep(3)



374
1
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=160&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
2
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=161&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
3
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=162&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
4
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=163&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
5
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=164&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
6
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=non

47
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=206&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
48
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=207&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
49
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=208&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
50
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=209&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
51
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=210&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
52
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=n

93
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=252&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
94
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=253&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
95
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=254&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
96
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=255&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
97
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=256&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
98
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=n

138
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=297&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
139
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=298&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
140
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=299&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
141
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=300&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
142
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=301&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
143
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&grou

183
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=342&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
184
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=343&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
185
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=344&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
186
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=345&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
187
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&group_by=none&page=346&filtertype_0=supportedlanguage&filter_relational_operator_0=equals&filter_0=en
188
https://openknowledge.worldbank.org/handle/10986/9387/discover?rpp=10&etal=0&grou

NoSuchElementException: Message: Unable to locate element: //*[@class='next-page-link']
